In [1]:
import pandas as pd
import numpy as np
import os
#os.chdir("../")

In [2]:
no_precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="reszta")
precambr_data = pd.read_excel(f"{os.getcwd()}\inputs\wucziapping_data.xlsx", sheet_name="Prekambr")

### NIE-PREKAMBR - SYSTEM ERY - JEDNO - ŁATWE

In [54]:
temp=no_precambr_data[["ERA", "SYSTEM"]].drop_duplicates()
n = len(temp)

In [60]:
target_list = temp["ERA"].unique()
target_list = [ele for ele in target_list for i in range(n*2)]
scope_list = temp["SYSTEM"].unique()

In [77]:
question_text = []
possible_answers = []
correct_answers = []

for target in target_list:
    question_text.append(f"Wypisz system ery {target}")
    correct_array = temp[temp["ERA"]==target]["SYSTEM"].unique()
    wrong_array = [element for element in scope_list if element not in correct_array]

    # print(f"CORRECT {correct_array}")
    # print(f"WRONG {wrong_array}")
    # if len(wrong_array)<4:
    #     print('WARNING!!!!!-------------------')
    #     possible_answers.append(np.repeat(np.nan, 5))
    #     correct_answers.append(np.nan)
    #     continue

    temp_possible_answers = np.concatenate(
        (np.random.choice(wrong_array, 3, replace=False), np.random.choice(correct_array, 1))
    )
    np.random.shuffle(temp_possible_answers)
    temp_possible_answers = np.append(temp_possible_answers, np.nan)
    for correct in temp_possible_answers:
        if correct in correct_array:
            correct_answers.append(correct)
            continue

    possible_answers.append(temp_possible_answers)

df1=pd.DataFrame({
    'question':question_text,
    'correct_answer':correct_answers
})

df2=pd.DataFrame(
    possible_answers,
    columns=["A", "B", "C", "D", "E"]
)

df_final = pd.concat([df1, df2], axis=1).drop_duplicates()

In [78]:
df_final

,question,correct_answer,A,B,C,D,E
0,Wypisz system ery kenozoik,czwartorzęd,czwartorzęd,kambr,karbon,trias,NaN
1,Wypisz system ery kenozoik,paleogen,ordowik,perm,paleogen,kambr,NaN
2,Wypisz system ery kenozoik,paleogen,perm,kreda,kambr,paleogen,NaN
3,Wypisz system ery kenozoik,paleogen,trias,jura,karbon,paleogen,NaN
4,Wypisz system ery kenozoik,paleogen,sylur,kreda,paleogen,jura,NaN
...,...,...,...,...,...,...,...
67,Wypisz system ery paleozoik,dewon,dewon,kreda,trias,czwartorzęd,NaN
68,Wypisz system ery paleozoik,dewon,kreda,neogen,paleogen,dewon,NaN
69,Wypisz system ery paleozoik,karbon,karbon,jura,paleogen,neogen,NaN
70,Wypisz system ery paleozoik,kambr,paleogen,trias,kambr,kreda,NaN


### MULTI

In [21]:
temp=no_precambr_data[["ERA", "SYSTEM"]].drop_duplicates()
n = len(temp)
target_list = temp["ERA"].unique()
target_list = [ele for ele in target_list for i in range(n*2)]
scope_list = temp["SYSTEM"].unique()

In [53]:
question_text = []
wrong_answers = []
correct_answers = []
possible_answers = []

for target in target_list:
    question_text.append(f"Wypisz system ery {target}")
    correct_array = temp[temp["ERA"]==target]["SYSTEM"].unique()
    wrong_array = [element for element in scope_list if element not in correct_array]

    # print(f"CORRECT {correct_array}")
    # print(f"WRONG {wrong_array}")
    
    # if len(wrong_array)<4:
    #     print('WARNING!!!!!-------------------')
    #     possible_answers.append(np.repeat(np.nan, 5))
    #     correct_answers.append(np.nan)
    #     continue

    if len(correct_array)>5:
        randomSampleCorrectSize = np.random.randint(1, 6)
    else:
        randomSampleCorrectSize = np.random.randint(1,len(correct_array))

    wrongSample = 5-randomSampleCorrectSize

    temp_wrong_answers = np.random.choice(wrong_array, wrongSample, replace=False)
    temp_correct_answers = np.random.choice(correct_array, randomSampleCorrectSize, replace=False)
    temp_possible_answers = np.concatenate((temp_correct_answers, temp_wrong_answers))
    np.random.shuffle(temp_possible_answers)
    possible_answers.append(temp_possible_answers)

    correct_answers.append(temp_correct_answers)

df1=pd.DataFrame({
    'question':question_text,
    'correct_answer':correct_answers
})

df2=pd.DataFrame(
    possible_answers,
    columns=["A", "B", "C", "D", "E"]
)

df_final = pd.concat([df1, df2], axis=1)

In [54]:
df_final

,question,correct_answer,A,B,C,D,E
0,Wypisz system ery kenozoik,"[neogen, czwartorzęd]",czwartorzęd,kambr,neogen,sylur,trias
1,Wypisz system ery kenozoik,[neogen],sylur,perm,kambr,karbon,neogen
2,Wypisz system ery kenozoik,"[paleogen, neogen]",paleogen,sylur,kreda,neogen,dewon
3,Wypisz system ery kenozoik,"[paleogen, neogen]",ordowik,trias,dewon,neogen,paleogen
4,Wypisz system ery kenozoik,"[neogen, czwartorzęd]",neogen,czwartorzęd,sylur,kreda,perm
...,...,...,...,...,...,...,...
67,Wypisz system ery paleozoik,"[ordowik, perm]",trias,jura,kreda,ordowik,perm
68,Wypisz system ery paleozoik,"[karbon, sylur, dewon, kambr]",dewon,karbon,sylur,czwartorzęd,kambr
69,Wypisz system ery paleozoik,[karbon],kreda,karbon,jura,neogen,czwartorzęd
70,Wypisz system ery paleozoik,"[kambr, ordowik]",jura,kambr,neogen,ordowik,paleogen


In [50]:
df2.reset_index()

,index,A,B,C,D,E
0,0,czwartorzęd,sylur,karbon,paleogen,kreda
1,1,paleogen,neogen,karbon,jura,kambr
2,2,ordowik,czwartorzęd,trias,perm,neogen
3,3,kambr,trias,karbon,czwartorzęd,dewon
4,4,jura,paleogen,ordowik,neogen,karbon
...,...,...,...,...,...,...
67,67,karbon,paleogen,perm,ordowik,kambr
68,68,kambr,dewon,jura,ordowik,perm
69,69,ordowik,sylur,karbon,kambr,perm
70,70,jura,perm,karbon,sylur,kambr


In [3]:
from questionBuilder import ClosedEndedQuestionBuilder
test = ClosedEndedQuestionBuilder(no_precambr_data, precambr_data)

In [4]:
df_single = test.prepare_single(False, False, 3, "ERA", "SYSTEM", "Wybierz system ery")

In [5]:
df_multi = test.prepare_multi(False, 3, "ERA", "SYSTEM", "Wybierz system ery")

In [6]:
df_multi

,question,correct_answer,A,B,C,D,E
0,Wybierz system ery kenozoik,[czwartorzęd],czwartorzęd,perm,grenland,trias górny,żywet
1,Wybierz system ery kenozoik,[neogen],neogen,artyńsk,kenozoik,chiban,mezozoik
2,Wybierz system ery kenozoik,[neogen],lochkow,noryk,prekambr,mezoproterozoik,neogen
3,Wybierz system ery kenozoik,[neogen],serrawal,plejstocen,neogen,koniak,drum
4,Wybierz system ery kenozoik,[czwartorzęd],szat,ordowik środkowy,dan,czwartorzęd,piętro 4
...,...,...,...,...,...,...,...
103,Wybierz system ery paleozoik,"[ordowik, sylur, kambr, karbon, dewon]",ordowik,dewon,sylur,kambr,karbon
104,Wybierz system ery paleozoik,"[perm, sylur]",sylur,zankl,jura górna,perm,paleogen
105,Wybierz system ery paleozoik,"[sylur, dewon, karbon, perm, kambr]",sylur,kambr,karbon,perm,dewon
106,Wybierz system ery paleozoik,"[karbon, dewon, ordowik]",walanżyn,karbon,ordowik,dewon,neoproterozoik


In [4]:
dfs = []
criteria = [
    # NON PRECAMBRIAN
    # era
    (False, False, 3, "ERA", "SYSTEM", "Wybierz system ery"),
    (False, True, 15, "ERA", "SYSTEM", "Wybierz system ery"),
    (False, False, 3, "ERA", "ODDZIAL", "Wybierz oddział ery"),
    (False, True, 15, "ERA", "ODDZIAL", "Wybierz oddział ery"),
    (False, False, 3, "ERA", "PIETRO", "Wybierz piętro ery"),
    (False, True, 15, "ERA", "PIETRO", "Wybierz piętro ery"),
    # system
    (False, False, 3, "SYSTEM", "ODDZIAL", "Wybierz oddział systemu"),
    (False, True, 15, "SYSTEM", "ODDZIAL", "Wybierz oddział systemu"),
    (False, False, 3, "SYSTEM", "PIETRO", "Wybierz piętro systemu"),
    (False, True, 15, "SYSTEM", "PIETRO", "Wybierz piętro systemu"),
    # oddział
    (False, False, 3, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału"),
    (False, True, 15, "ODDZIAL", "PIETRO", "Wybierz piętro oddziału"),

    ## PRECAMBRIAN
    # prekambr
    #(True, False, 2, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru"),
    (True, True, 10, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru"),
    (True, False, 2, "Jednostka nieformalna", "Era", "Prekambr - Wybierz erę prekambru"),
    (True, True, 10, "Jednostka nieformalna", "Era", "Prekambr - Wybierz erę prekambru"),
    (True, False, 2, "Jednostka nieformalna", "System", "Prekambr - Wybierz system prekambru"),
    (True, True, 10, "Jednostka nieformalna", "System", "Prekambr - Wybierz system prekambru"),
    # eon
    (True, False, 2, "Eon", "Era", "Prekambr - Wybierz erę enou"),
    (True, True, 10, "Eon", "Era", "Prekambr - Wybierz erę enou"),
    (True, False, 2, "Eon", "System", "Prekambr - Wybierz system enou"),
    (True, True, 10, "Eon", "System", "Prekambr - Wybierz system enou"),
    # system ery
    (True, False, 2, "Era", "System", "Prekambr - Wybierz system ery"),
    (True, True, 10, "Era", "System", "Prekambr - Wybierz system ery"),

]
n = 0
for is_prek, is_hard, coeff, key1, key2, quest_txt in criteria:
    temp = test.prepare_single(is_prek, is_hard, coeff, key1, key2, quest_txt)
    dfs.append(temp)
    n+=1
    print(n)
    print(f"{is_prek} {is_hard} {coeff} {key1}, {key2}, {quest_txt}")

12
1
False False 3 ERA, SYSTEM, Wybierz system ery
172
2
False True 15 ERA, SYSTEM, Wybierz system ery
34
3
False False 3 ERA, ODDZIAL, Wybierz oddział ery
172
4
False True 15 ERA, ODDZIAL, Wybierz oddział ery
102
5
False False 3 ERA, PIETRO, Wybierz piętro ery
172
6
False True 15 ERA, PIETRO, Wybierz piętro ery
34
7
False False 3 SYSTEM, ODDZIAL, Wybierz oddział systemu
172
8
False True 15 SYSTEM, ODDZIAL, Wybierz oddział systemu
102
9
False False 3 SYSTEM, PIETRO, Wybierz piętro systemu
172
10
False True 15 SYSTEM, PIETRO, Wybierz piętro systemu
102
11
False False 3 ODDZIAL, PIETRO, Wybierz piętro oddziału
172
12
False True 15 ODDZIAL, PIETRO, Wybierz piętro oddziału
172
13
True True 10 Jednostka nieformalna, Eon, Prekambr - Wybierz eon prekambru
8
WARNING!!!!!-------------------
[]
3


ValueError: 

In [4]:
c = (True, True, 5, "Jednostka nieformalna", "Eon", "Prekambr - Wybierz eon prekambru")
df1 = test.prepare_single(c[0], c[1], c[2], c[3], c[4], c[5])

['aalen' 'aeron' 'akwitan' 'alb' 'anizyk' 'apt' 'archaik' 'artyńsk'
 'assel' 'bajos' 'barrem' 'barton' 'baszkir' 'baton' 'berrias' 'brak'
 'burdygał' 'cenoman' 'chiban' 'cisural' 'czangsing' 'czwartorzęd' 'dan'
 'daping' 'darriwil' 'dewon' 'dewon dolny' 'dewon górny' 'dewon środkowy'
 'drum' 'dżangszan' 'ediakar' 'eifel' 'ektas' 'ems' 'eoarchaik' 'eocen'
 'epoka 2' 'famen' 'floj' 'fortun' 'fran' 'furong' 'gelas' 'gorst'
 'grenland' 'gużang' 'gwadelup' 'gżel' 'hadeik' 'hettang' 'hirnant'
 'holocen' 'homer' 'hoteryw' 'ind' 'iprez' 'jura' 'jura dolna'
 'jura górna' 'jura środkowa' 'kalabr' 'kalim' 'kambr' 'kampan' 'kapitan'
 'karbon' 'karnik' 'kasim' 'kat' 'kelowej' 'kenozoik' 'kimeryd' 'koniak'
 'kreda' 'kreda dolna' 'kreda górna' 'kriogen' 'kungur' 'ladyn' 'landower'
 'lang' 'lochkow' 'loping' 'ludford' 'ludlow' 'lutet' 'mastrycht'
 'megalaj' 'messyn' 'mezoarchaik' 'mezoproterozoik' 'mezozoik' 'miaoling'
 'miocen' 'mississip' 'moskow' 'neoarchaik' 'neogen' 'neoproterozoik'
 'northgrip' 

In [5]:
df1

,question,correct_answer,A,B,C,D,E
0,Prekambr - Wybierz eon prekambru prekambr,archaik,hirnant,archaik,kreda górna,dan,tremadok
1,Prekambr - Wybierz eon prekambru prekambr,hadeik,hadeik,paleoproterozoik,gużang,ems,chiban
2,Prekambr - Wybierz eon prekambru prekambr,archaik,artyńsk,northgrip,road,sakmar,archaik
3,Prekambr - Wybierz eon prekambru prekambr,archaik,trias środkowy,pensylwan,zankl,archaik,mississip
4,Prekambr - Wybierz eon prekambru prekambr,proterozoik,cisural,piętro 2,proterozoik,kreda górna,gużang
5,Prekambr - Wybierz eon prekambru prekambr,archaik,sten,orosir,plejstocen,gorst,archaik
6,Prekambr - Wybierz eon prekambru prekambr,archaik,piętro 4,synemur,loping,lutet,archaik
7,Prekambr - Wybierz eon prekambru prekambr,archaik,priabon,kreda,stater,alb,archaik
8,Prekambr - Wybierz eon prekambru prekambr,proterozoik,piacent,mezoarchaik,fortun,chiban,proterozoik
9,Prekambr - Wybierz eon prekambru prekambr,hadeik,santon,hadeik,aalen,baszkir,paleocen


In [5]:
# df_easy = test.prepare_single(False, False, 3, "ERA", "SYSTEM", "Wybierz system ery")
# df_hard = test.prepare_single(False, True, 5, "ERA", "SYSTEM", "Wybierz system ery")

,question,correct_answer,A,B,C,D,E
0,Wybierz system ery kenozoik,neogen,trias,neogen,kreda,perm,NaN
1,Wybierz system ery kenozoik,czwartorzęd,dewon,czwartorzęd,kambr,jura,NaN
2,Wybierz system ery kenozoik,czwartorzęd,dewon,karbon,czwartorzęd,kambr,NaN
3,Wybierz system ery kenozoik,neogen,jura,neogen,sylur,kambr,NaN
4,Wybierz system ery kenozoik,czwartorzęd,dewon,perm,czwartorzęd,kreda,NaN
...,...,...,...,...,...,...,...
103,Wybierz system ery paleozoik,ordowik,jura,czwartorzęd,ordowik,neogen,NaN
104,Wybierz system ery paleozoik,sylur,czwartorzęd,paleogen,jura,sylur,NaN
105,Wybierz system ery paleozoik,kambr,kreda,trias,paleogen,kambr,NaN
106,Wybierz system ery paleozoik,ordowik,neogen,ordowik,jura,czwartorzęd,NaN


In [6]:
df_hard

,question,correct_answer,A,B,C,D,E
0,Wybierz system ery kenozoik,neogen,kelowej,tyton,neoarchaik,żywet,neogen
1,Wybierz system ery kenozoik,czwartorzęd,walanżyn,wenlok,czwartorzęd,megalaj,aeron
2,Wybierz system ery kenozoik,neogen,sandb,mezoproterozoik,neogen,trias środkowy,serrawal
3,Wybierz system ery kenozoik,neogen,sider,floj,neoarchaik,neogen,kapitan
4,Wybierz system ery kenozoik,neogen,kalabr,neogen,kreda dolna,terenew,serpuchow
...,...,...,...,...,...,...,...
175,Wybierz system ery paleozoik,ordowik,word,mississip,ediakar,synemur,ordowik
176,Wybierz system ery paleozoik,kambr,kambr,northgrip,kenozoik,kelowej,mezoproterozoik
177,Wybierz system ery paleozoik,perm,hadeik,perm,kimeryd,gżel,kreda górna
178,Wybierz system ery paleozoik,perm,ind,lang,perm,zeland,wizen


In [12]:
data = no_precambr_data
dataPrekambr = precambr_data
isPrekambr = True
isHard = False
startColIdx = 1
endColIdx = 3
multiplyRowsIdx = 2
colName1 = "Eon"
colName2 = "System"
questionPattern = "Prekambr - wybierz system ery"

if isPrekambr:
    df = dataPrekambr.iloc[:, [startColIdx,endColIdx]]
else:
    df = data.iloc[:, [startColIdx,endColIdx]]

df = df.loc[df.index.repeat(multiplyRowsIdx)]

random_answers = []
questions = []

if isHard:
    scope = np.unique(
            np.concatenate((data["PIETRO"].unique(), data["ODDZIAL"].unique(), data["SYSTEM"].unique(), data["ERA"].unique(), 
                            dataPrekambr["Jednostka nieformalna"].unique(), dataPrekambr["Eon"].unique(), 
                            dataPrekambr["Era"].unique(), dataPrekambr["System"].unique()))
                            )

else:
    if isPrekambr:
        scope = dataPrekambr[colName2].unique()
    else:
        scope = data[colName2].unique()

print(scope)

for i in df[colName1]:
    
    if isPrekambr:
        temp_correct = dataPrekambr[dataPrekambr[colName1]==i][colName2].array
    else:
        temp_correct = data[data[colName1]==i][colName2].array

    indexes = np.nonzero(np.in1d(scope, temp_correct))[0]

    print(temp_correct)

    temp_scope = np.delete(scope, indexes)
    
    print(temp_scope)

    if isHard:
        temp = np.concatenate((np.random.choice(temp_scope,4), np.random.choice(temp_correct,1)))
    else:
        temp = np.concatenate((np.random.choice(temp_scope,3), np.random.choice(temp_correct,1)))

    np.random.shuffle(temp)

    if not isHard:
        temp = np.append(temp, np.nan)

    random_answers.append(temp)
    questions.append(f"{questionPattern} {i}")

p1 = pd.DataFrame(questions, columns=["question"])
p2 = pd.DataFrame(random_answers, columns=["A", "B", "C", "D", "E"]).drop_duplicates()
p3 = pd.concat([p1, p2], axis=1)

all_correct_answers = []

for index, row in p3.iterrows():
    temp_val1 = row["question"].split(" ")[-1]

    if temp_val1.isdigit() or temp_val1 in ("górna", "dolna", "środkowa", "górny", "środkowy", "dolny"):
        temp_val1 = row["question"].split(" ")[-2:]
        temp_val1 = " ".join(temp_val1)

    temp_abcde = row[["A", "B", "C", "D", "E"]].tolist()

    if isPrekambr:
        correct_answers = dataPrekambr[dataPrekambr[colName1]==temp_val1][colName2].tolist()
    else:
        correct_answers = data[data[colName1]==temp_val1][colName2].tolist()

    for answer in temp_abcde:
        if answer in correct_answers:
            all_correct_answers.append(answer)
            break

p4 = pd.concat([p3, pd.DataFrame(all_correct_answers, columns=["correct_answer"])], axis=1)

['ediakar' 'kriogen' 'ton' 'sten' 'ektas' 'kalim' 'stater' 'orosir' 'riak'
 'sider' 'brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'sten',   'ektas',   'kalim',  'stater',
  'orosir',    'riak',   'sider']
Length: 10, dtype: object
['brak']
<NumpyExtensionArray>
['ediakar', 'kriogen',     'ton',    'ste

In [6]:
p4

,question,A,B,C,D,E,correct_answer
0,Prekambr - wybierz system ery proterozoik,brak,brak,brak,orosir,NaN,orosir
1,Prekambr - wybierz system ery proterozoik,brak,brak,ton,brak,NaN,ton
2,Prekambr - wybierz system ery proterozoik,brak,brak,riak,brak,NaN,riak
3,Prekambr - wybierz system ery proterozoik,brak,brak,orosir,brak,NaN,orosir
4,Prekambr - wybierz system ery proterozoik,brak,brak,brak,sten,NaN,sten
5,Prekambr - wybierz system ery proterozoik,brak,brak,brak,ektas,NaN,ektas
6,Prekambr - wybierz system ery proterozoik,brak,brak,ektas,brak,NaN,ektas
7,Prekambr - wybierz system ery proterozoik,NaN,NaN,NaN,NaN,NaN,stater
8,Prekambr - wybierz system ery proterozoik,stater,brak,brak,brak,NaN,ektas
9,Prekambr - wybierz system ery proterozoik,ektas,brak,brak,brak,NaN,sider
